In [1]:
from tqdm import tqdm
from glob import glob
import numpy as np
import os
from utils.process_data import split_train_val, get_data_properties,make_dirs
from utils.generate_crops import *
from multiprocessing import Pool

import json

In [6]:
data_dir = '../Data'
project_name = '20220715'
if os.path.exists(os.path.join(data_dir,project_name)):
    print("`{}` is choosen as datapath".format(os.path.join(data_dir,project_name)))

`../Data/20220715` is choosen as datapath


In [4]:
# make_dirs(data_dir,project_name)

In [5]:
split_train_val(data_dir,project_name,subset=0.15)

../Data/test/classmaps
24 24
24
Number of samples found in '../Data/test/images' equal to 24


In [7]:
data_properties_dir = get_data_properties(data_dir, project_name, train_val_name=['train', 'val'])


0it [00:00, ?it/s]


Number of instances per class
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}


100%|██████████| 24/24 [00:04<00:00,  5.14it/s]


Foreground weight of the `20220715` dataset set equal to '{1.0: 7.373971834221564, 2.0: 7.254255756783858, 3.0: 5.722661555739282, 4.0: 9.427849786431093, 5.0: 9.355005395320262}'


100%|██████████| 24/24 [00:40<00:00,  1.68s/it]


Minimum object size of the `20220715` dataset is equal to 216
Mean object size of the `20220715` dataset is equal to 8441.241433021807
Maximum object size of the `20220715` dataset is equal to 33901
Average object size of the `20220715` dataset along `x` is equal to 105.045
Std. dev object size of the `20220715` dataset along `x` is equal to 60.974
Average object size of the `20220715` dataset along `y` is equal to 115.193
Std. dev object size of the `20220715` dataset along `y` is equal to 71.614


100%|██████████| 24/24 [00:00<00:00, 24.16it/s]


Tile size of the `20220715` dataset set equal to (1024, 1024)


In [8]:
# n_sigma is used to control the size of crop
n_sigma = 5


def round_up_8(x):
    return (x.astype(int)+7) & (-8)


crops_dir = '../Data/augmented15'
data_subsets = ['train', 'val'] 
crop_size =416
# crop_size = np.maximum(round_up_8(data_properties_dir['avg_object_size_y'] + n_sigma*data_properties_dir['stdev_object_size_y']),
# round_up_8(data_properties_dir['avg_object_size_x'] + n_sigma*data_properties_dir['stdev_object_size_x']))
print("Crop size in x and y will be set equal to {}".format(crop_size))



Crop size in x and y will be set equal to 416


In [9]:
import itertools


for data_subset in data_subsets:
    image_dir = os.path.join(data_dir, project_name, data_subset, 'images')
    instance_dir = os.path.join(data_dir, project_name, data_subset,'instances')
    classmap_dir = os.path.join(data_dir, project_name, data_subset,'classmaps')
    hs_dir = os.path.join(data_dir, project_name, data_subset,'hs')
    
    
    
    image_names = sorted(glob.glob(os.path.join(image_dir, '*.png'))) 
    instance_names = sorted(glob.glob(os.path.join(instance_dir, '*.png')))
    classmap_names = sorted(glob.glob(os.path.join(classmap_dir, '*.png')))
    hs_names = sorted(glob.glob(os.path.join(hs_dir, '*.npy')))
#     header_names = sorted(glob.glob(os.path.join(hs_dir, '*.hdr')))
    crop_size_iter = list(itertools.repeat(crop_size,len(image_names)))
    crop_dir_iter = list(itertools.repeat(os.path.join(crops_dir,'H2giga'),len(image_names)))
    type_iter = list(itertools.repeat(data_subset,len(image_names)))

    
#     item = zip(image_names,instance_names,classmap_names,hs_names,header_names,crop_size_iter,crop_dir_iter,type_iter)
    item = zip(image_names,instance_names,classmap_names,hs_names,crop_size_iter,crop_dir_iter,type_iter)
    
    
    with Pool(2) as p:
        print("inside")
        max_ = len(image_names)
        with tqdm(total=max_) as pbar:
            for _ in p.imap_unordered(process_with_pool, item):
                pbar.update()
        # shutdown the process pool
        p.close()
        # wait for all issued task to complete
        p.join()

inside


  0%|          | 0/21 [00:00<?, ?it/s]

*******strating***********strating****



  5%|▍         | 1/21 [00:35<11:54, 35.71s/it]

*******strating****


...

  0%|          | 0/3 [00:00<?, ?it/s]

*******strating***********strating****



 33%|███▎      | 1/3 [00:38<01:16, 38.44s/it]

*******strating****


100%|██████████| 3/3 [01:17<00:00, 25.76s/it]
